In [2]:
from scipy.optimize import linprog
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt

from gurobi import *

## Define situation variables

In [3]:
I = 2 # number of resources
k = 2 * np.ones(I,np.int) # resource utilization times
J = 2 # number of agents
d = 2 # wait time for each agent
T = J+d # total time for matches to occur

## Make primal constraint matrix

In [11]:
def in_constraint(v):
    if v[1]:
        return True
    else:
        return False

def print_tableau(matches):
    for i in range(matches.shape[2]):
        print(matches[:,:,i])
        
def create_tableau(I,J,T):
    valid_matches = np.zeros((I,J,T), np.int)

    #valid allocations based on arival time
    for t in range(T):
        valid_matches[:,max(0,t-d):min(t+1,J),t] = 1

    #make weights uniform across time
    pairing_weights = np.random.random(valid_matches.shape) 
    for t in range(1,pairing_weights.shape[2]):
        pairing_weights[:,:,t] = pairing_weights[:,:,0]
    
    pairing_weights[valid_matches == 0] = 0

    return valid_matches, pairing_weights
    

In [323]:
def primal_constraint_matrix(valid_matches, I,J,T):

    constraints = np.zeros((valid_matches.sum()+J,valid_matches.size))
    cix = 0
    #constraints limiting to one resource allocation in the time interval
    for i in range(I):
        for t in range(T):
            constraint = np.zeros((I,J,T), np.int)
            endix = min(t+k[i],T)
            constraint[i,:,t:endix] = 1 
            constraints[cix,:] = constraint.reshape((1, constraint.shape[0] * constraint.shape[1] * constraint.shape[2]))
            cix += 1

    #constraints limiting each agent to only match once            
    for j in range(J):
        constraint = np.zeros((I,J,T), np.int)
        valid_mask = constraint.copy()
        valid_mask[:,j,:] = 1

        constraint[(valid_matches == 1) & (valid_mask ==1)] = 1
        constraints[cix+j,:] = constraint.reshape((1, constraint.shape[0] * constraint.shape[1] * constraint.shape[2]))
    
    return constraints

### Dual constraint matrix

To make matrix: 
- Create a constraint map to see which alphas/betas apply at a given location in the primal. Each valid location will correspond to a constraint in the dual, and the variables == 1 will be those in the `cmap[i][j][t]`

In [35]:
constraint_map = np.zeros((I,J,T,T*I+J), np.int)
cix = 0

#constraints limiting to one resource allocation in the time interval
for i in range(I):
    for t in range(T):
        constraint = np.zeros((I,J,T), np.int)
        endix = min(t+k[i],T)
        constraint[i,:,t:endix] = 1 
        constraint_map[:,:,:,cix] = constraint.copy()
        cix += 1

#constraints limiting each agent to only match once            
for j in range(J):
    constraint = np.zeros((I,J,T), np.int)
    valid_mask = constraint.copy()
    valid_mask[:,j,:] = 1
    constraint[(valid_matches == 1) & (valid_mask ==1)] = 1
    constraint_map[:,:,:,cix] = constraint.copy()
          

In [43]:
constraint_map.shape
valid_matches.sum()

12

In [41]:
constraint_map[0,0,0,:].shape

(10,)

In [38]:
dual_constraint_matrix = np.zeros(valid_matches.sum(), constraint_map.shape[3])

cix = 0
for i in range(I):
    for j in range(J):
        for t in range(T):
            if valid_matches[i][j][t]:
                dual_constraint_matrix[cix,:] = constraint_map 
                constraint = constraint_map[i,j,t,:]



In [30]:
cmap = []
for i in range(I):
    cmap.append([])
    for j in range(J):
        cmap[i].append([])
        for t in range(T):
            cmap[i][j].append([])

cix = 0
#constraints limiting to one resource allocation in the time interval
for i in range(I):
    for t in range(T):
        endix = min(t+k[i],T) 
        for tp in range(t,endix):
            for j in range(J):
                if valid_matches[i][j][tp]:
                    cmap[i][j][tp] = cix
        cix += 1
            

# #constraints limiting each agent to only match once            
# for j in range(J):
#     constraint = np.zeros((I,J,T), np.int)
#     valid_mask = constraint.copy()
#     valid_mask[:,j,:] = 1

#     constraint[(valid_matches == 1) & (valid_mask ==1)] = 1
#     constraints[cix+j,:] = constraint.reshape((1, constraint.shape[0] * constraint.shape[1] * constraint.shape[2]))



In [23]:
constraint_map

array([[[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]],

        [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0, 1, 0],
         [0, 1, 1, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 1, 1, 0, 0, 0, 0, 1, 0]]],


       [[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 1, 0, 0]],

        [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 1, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 1, 1, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 1, 1, 1, 0]]]])

In [13]:
valid_matches, pairing_weights = create_tableau(I,J,T)


In [372]:
def primal_solutions(pairing_weights, I, J, T):
    m = Model("dynamicmatch_primal")
    m.modelSense = GRB.MAXIMIZE
    m.setParam( 'OutputFlag', True )

    weights = pairing_weights.reshape(pairing_weights.shape[0] * pairing_weights.shape[1] * pairing_weights.shape[2])
    c = -1 * pairing_weights.reshape(pairing_weights.shape[0] * pairing_weights.shape[1] * pairing_weights.shape[2])
    constraints = primal_constraint_matrix(valid_matches, I,J,T)


    keys = range(constraints.shape[1])
    variables = m.addVars(keys,
                    vtype=GRB.CONTINUOUS,
                     obj=weights,
                     name="primal",
                     lb=0)

    for constraint in constraints:
        m.addConstr(sum(variables[o]*c for o,c in filter(in_constraint, zip(variables,constraint))) <= 1)



    m.optimize()
    allocations = np.array([variables[var].X for var in variables]).reshape(pairing_weights.shape)
    # m.write('constraint.lp')

    return m.objVal, allocations


In [373]:
obj, allocs = primal_solutions(pairing_weights, I, J, T)

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 3150 rows, 18125 columns and 87500 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-04, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 2425 rows and 15000 columns
Presolve time: 0.11s
Presolved: 725 rows, 3125 columns, 18225 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 7.900e+03
 Factor NZ  : 1.827e+04 (roughly 2 MBytes of memory)
 Factor Ops : 4.758e+05 (less than 1 second per iteration)
 Threads    : 1

Barrier performed 0 iterations in 0.14 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 433 iterations and 0.15 seconds
Optimal objective  2.410413828e+01
